In [ ]:
# 2022A 2) 第一问结果

In [1]:
"""
问题	入射波浪频率 (s-1)	垂荡附加质量 (kg)	纵摇附加转动惯量 (kg·m2)	垂荡兴波阻尼系数 (N·s/m)	纵摇兴波阻尼系数 (N·m·s)	垂荡激励力振幅 (N)	纵摇激励力矩振幅 (N·m)
问题1	1.4005	1335.535	6779.315	656.3616	151.4388	6250	1230
问题2	2.2143	1165.992	7131.29	    167.8395	2992.724	4890	2560
问题3	1.7152	1028.876	7001.914	683.4558	654.3383	3640	1690
问题4	1.9806	1091.099	7142.493	528.5018	1655.909	1760	2140
"""
pass

In [2]:
"""
参数	取值
浮子质量 (kg)	4866
浮子底半径 (m)	1
浮子圆柱部分高度 (m)	3
浮子圆锥部分高度 (m)	0.8
振子质量 (kg)	2433
振子半径 (m)	0.5
振子高度 (m)	0.5
海水的密度 (kg/m3)	1025
重力加速度 (m/s2)	9.8
弹簧刚度 (N/m)	80000
弹簧原长 (m)	0.5
扭转弹簧刚度 (N·m)	250000
静水恢复力矩系数 (N·m)	8890.7
"""
pass

In [3]:
import numpy as np

In [4]:
rho = 1025
g = 9.8
rho_g_pi_r2 = rho * g * np.pi * (1)**2

In [5]:
wave_fre = 2.2143
wave_force_max = 4890
k = 80000
m_oscillator = 2433
m_floater = 4866
m_floater_extra = 1335.535
wave_gamma = 167.8395

In [6]:
time = 40 * 2 * np.pi / wave_fre
dt = 1e-3
n_ = int(time / dt) + 1

In [7]:
def output(gamma):
    ret = 0
    ksi = np.zeros(n_)
    x = np.zeros(n_)
    # -k(x - ksi) - gamma(D x - D ksi) = m_oscillator D2 x
    # -k(x - ksi) - gamma(D x - D ksi) - rho g pi r2 ksi + f cos omega t =  (m_floater + m_floater_extra) D2 ksi
    for t_ in range(2, n_):
        # -k(x - ksi)[t_] - gamma (x - ksi)([t_] - [t_ - 1]) / dt = m (x[t_+1] - 2 x[t_] + x[t_-1]) / dt 2
        spring_force = -k * (x[t_ - 1] - ksi[t_ - 1])
        relative_speed = ((x[t_ - 1] - ksi[t_ - 1]) - (x[t_ - 2] - ksi[t_ - 2])) / dt
        damping_force = -gamma * relative_speed
        ret += np.abs(damping_force * relative_speed * dt)
        x[t_] = 2 * x[t_ - 1] - x[t_ - 2] + (spring_force + damping_force) * dt ** 2 / m_oscillator
        floating_force = -rho_g_pi_r2 * ksi[t_ - 1]
        wave_force = wave_force_max * np.cos(wave_fre * t_ * dt)
        secondary_wave_force = -wave_gamma * (ksi[t_ - 1] - ksi[t_ - 2]) / dt
        ksi[t_] = 2 * ksi[t_ - 1] - ksi[t_ - 2] + (secondary_wave_force + floating_force + wave_force - spring_force - damping_force) * dt ** 2 / (m_floater + m_floater_extra)
    return ret

In [8]:
output(40000)

24085.98993429681

In [9]:
ans = 37281.36774448058
out = output(ans)

In [10]:
# simulated annealing
T_sim = 10
T_step = 0.1
iterations = int((T_sim - 1) / T_step)
sigma = 1

for i in range(iterations):
    next_ans = ans + np.random.normal(0, sigma)
    next_ans = np.clip(next_ans, 0, 100000)
    next_out = output(next_ans)
    if next_out >= out or np.random.rand() < np.exp((next_out - out) / T_sim):
        ans = next_ans
        out = next_out
    if i % int(iterations / 10) == 0:
        print(f"""
        iteration: {i}
        ans: {ans}
        out: {out}
            """)


        iteration: 0
        ans: 37281.06778358715
        out: 24148.400579971265
            

        iteration: 9
        ans: 37281.03145430287
        out: 24148.40066273799
            

        iteration: 18
        ans: 37274.197295569684
        out: 24148.41585097177
            

        iteration: 27
        ans: 37271.87132451589
        out: 24148.420845679964
            

        iteration: 36
        ans: 37270.45710551035
        out: 24148.42383921296
            

        iteration: 45
        ans: 37272.754249050275
        out: 24148.418960159517
            

        iteration: 54
        ans: 37272.5487816556
        out: 24148.419400079674
            

        iteration: 63
        ans: 37275.15885353732
        out: 24148.41376025096
            

        iteration: 72
        ans: 37279.54430463071
        out: 24148.40403290134
            

        iteration: 81
        ans: 37282.35000637162
        out: 24148.397644380355
            


In [11]:
ans

37284.080241718904